In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import qutip
import theano
import theano.tensor as T
from collections import OrderedDict
import itertools
from ../QubitNetwork import QubitNetwork
from qubit_network import load_network_from_file, sgd_optimization, transfer_J_values
from utils import *
from results_analysis_utils import *

pairs = list(itertools.combinations(range(4), 2))

SyntaxError: invalid syntax (<ipython-input-3-d11cdfe98637>, line 12)

## Test with interactions from *Banchi et al.*.
The Fredkin gate is indeed reproduced with good fidelity, and regardless of the initial ancilla state.

In [ ]:
net_paper = QubitNetwork(
    num_qubits=4,
    system_qubits=3,
    interactions=('all', ['zz', 'xx', 'yy', 'z', 'x']),
    ancillae_state=qutip.basis(2, 0),
    target_gate=qutip.fredkin()
)
net_paper.J.set_value(np.zeros_like(net_paper.J.get_value()))

Jvalues = net_paper.J.get_value()

J01xx = 13.6
J02xx = J01xx

J12xx = -4.712
J12yy = J12zz = J12xx

J13xx = 8.4
J23xx = J13xx

J01zz = 11.15
J02zz = J01zz

h3x = 1.025
h0z = np.pi

Jvalues[net_paper.tuple_to_J_index(((0, 1), 'xx'))] = J01xx / 4
Jvalues[net_paper.tuple_to_J_index(((0, 2), 'xx'))] = J02xx / 4
Jvalues[net_paper.tuple_to_J_index(((1, 2), 'xx'))] = J12xx / 4
Jvalues[net_paper.tuple_to_J_index(((1, 2), 'yy'))] = J12yy / 4
Jvalues[net_paper.tuple_to_J_index(((1, 2), 'zz'))] = J12zz / 4
Jvalues[net_paper.tuple_to_J_index(((1, 3), 'xx'))] = J13xx / 4
Jvalues[net_paper.tuple_to_J_index(((2, 3), 'xx'))] = J23xx / 4
Jvalues[net_paper.tuple_to_J_index(((0, 1), 'zz'))] = J01zz / 4
Jvalues[net_paper.tuple_to_J_index(((0, 2), 'zz'))] = J02zz / 4
Jvalues[net_paper.tuple_to_J_index((0, 'z'))] = h0z / 2
Jvalues[net_paper.tuple_to_J_index((3, 'x'))] = h3x / 2

net_paper.J.set_value(Jvalues)

# net_paper.test_fidelity(n_samples=1000)
gate = qutip.Qobj(net_paper.get_current_gate(), dims=[[2, 2, 2, 2]] * 2)
gate = chop(gate).ptrace([0, 1, 2])
gate = gate * np.exp(-1j * np.angle(gate[0, 0]))
gate = gate / gate[0, 0]
chop(gate, 0.01)